# Import packages

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import xgboost as xgb
import random
import os
from sklearn.metrics import auc, precision_recall_curve
from sklearn.preprocessing import MinMaxScaler
import pickle

# Define functions

In [31]:
def create_relative_annotation(genes):
    for feature in genes.columns:
        if len(genes) < 1:
            continue
        if "rel_" in feature:
            continue
        if "distance" in feature:
            maxval = max(genes[feature].astype(float))
            minval = min(genes[feature].astype(float))
            if maxval == minval:
                genes["rel_" + feature] = 1
            else:
                genes["rel_" + feature] = maxval - genes[feature]
                genes["rel_" + feature] = genes["rel_" + feature] / float(
                    maxval - minval
                )
        elif any(x in feature for x in ["max", "sum", "PoPS", "MAGMA_Z"]):
            maxval = max(genes[feature].astype(float))
            minval = min(genes[feature].astype(float))
            if maxval == minval and abs(maxval) > 0.0:
                genes["rel_" + feature] = 1
            elif maxval == 0.0:
                genes[f"rel_{feature}"] = 0
            else:
                genes[f"rel_{feature}"] = (genes[feature].astype(float) - minval) / (maxval - minval)
    return genes

# Train FLAMES (LOCO) models

In [33]:
#load training data
df = pd.read_csv(os.path.join('annotated_loci', 'all_FLAMES.csv'))
df = df[df['type'] == 'protein_coding']
df = df.fillna(0)

In [34]:
feats = ['rel_MAGMA_Z', 'VEP_max', 'rel_VEP_max', 'CADD_sum', 'rel_CADD_sum', 'CLPP_GTEX_tissue_weighted_eQTL_sum', 'rel_CLPP_GTEX_tissue_weighted_eQTL_sum', 'RoadmapEpi_sum', 'rel_RoadmapEpi_sum', 'HACER_PRO-seq_GRO-seq_sum', 'rel_HACER_PRO-seq_GRO-seq_sum', 'Promoter_sum', 'rel_Promoter_sum', 'F5_ep_sum', 'rel_F5_ep_sum', 'Jung_HiC_sum', 'rel_Jung_HiC_sum', 'GeneHancer_sum', 'rel_GeneHancer_sum', 'EpiMap_sum', 'rel_EpiMap_sum', 'mean_CLPP_eQTL_eQTLCatalog_max', 'rel_mean_CLPP_eQTL_eQTLCatalog_max', 'ABC_CRISPR_sum', 'rel_ABC_CRISPR_sum', 'ABC_EP_sum', 'rel_ABC_EP_sum', 'max_CLPP_rQTL_eQTLCatalog_max', 'rel_max_CLPP_rQTL_eQTLCatalog_max', 'Javierre_HiC_sum', 'rel_Javierre_HiC_sum', 'HACER_CAGE_max', 'rel_HACER_CAGE_max', 'CLPP_eQTLgen_sum', 'rel_CLPP_eQTLgen_sum', 'cicero_sum', 'rel_cicero_sum', 'MAGMA_Z', 'rel_weighted_distance', 'rel_weighted_TSS_distance', 'weighted_distance']
df = df.groupby('filename').apply(create_relative_annotation)
df= df.reset_index(drop=True)
traindf = df.copy()

In [ ]:
#Train LOCO models
LOCO_models = []
sav_params = []
iters = []
for chrom in range(1,23):    
    keeps = set(df[df['chr'] != chrom]['filename'])
    #train in LOCO framework
    if len(keeps) < 100:
        extra = set(df[df['chr'] != 23-chrom]['filename'])
        if len(extra) + len(keeps) >= 100:
            keeps.append(extra[0:100-len(keeps)])
        else:
            keeps.append(extra)
    
                  
    X_train = df[df['filename'].isin(keeps)][feats]
    y_train = df[df['filename'].isin(keeps)]['TP']
    
    X_valid = df[~ df['filename'].isin(keeps)][feats]
    y_valid = df[~ df['filename'].isin(keeps)]['TP']
    eval_set = [(X_train, y_train), (X_valid, y_valid)]
    
    # Define the parameter grid for random search
    param_dist = {
        
        "learning_rate": [0.1],
        'max_depth': [3, 4, 5],
        'colsample_bytree': [0.8,0.9],
        'min_child_weight': [3, 4, 5]
        # Add more parameters as needed
    }

    # Instantiate the XGBoost classifier
    clf = xgb.XGBClassifier(objective='binary:logistic', seed=42, eval_metric='rmse', early_stopping_rounds=7)

    # Perform random search
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42, verbose =False)
        
    fit_params={ 
            "eval_set" : [[X_valid, y_valid]],
                "verbose": False
               }
    
    random_search.fit(X_train, y_train, **fit_params)

    # Print the best parameters found
    print("Best parameters found: ", random_search.best_params_)

    # Get the best estimator
    best_model = random_search.best_estimator_
    num_trees = best_model.best_iteration

    # Fit the best model on the training data with early stopping
    
    best_model.fit(X_train, y_train, eval_set=eval_set, verbose=True)
  
    # Evaluate the best model on test data
    LOCO_models.append(best_model)
    sav_params.append(random_search.best_params_)
    iters.append(num_trees)

    print(f'Finnished chrom {chrom}')
print('done')

final_params = {}
for key in sav_params[0]:
    final_params[key] = round(np.mean([x[key] for x in sav_params]),2)

In [ ]:
# #Train consensus
random_locs = list(set(traindf['filename']))
random.seed(len('FLAMES reproducibility'))
random.shuffle(random_locs)
holdout = random_locs[0:int(len(set(traindf['filename']))/9)]

traindf = df
filename_counts = traindf['filename'].value_counts()


X_train = traindf[~traindf['filename'].isin(holdout)][feats]
y_train = traindf[~traindf['filename'].isin(holdout)]['TP']
X_valid = traindf[traindf['filename'].isin(holdout)][feats]
y_valid = traindf[traindf['filename'].isin(holdout)]['TP']

eval_set = [(X_train, y_train), (X_valid, y_valid)]
    
FLAMES_xgb = xgb.XGBClassifier(objective='binary:logistic', seed=42, min_child_weight=int(np.round(final_params['min_child_weight'],0)), max_depth=int(np.round(final_params['max_depth'],0)), learning_rate=final_params['learning_rate'], colsample_bytree=final_params['colsample_bytree'],  eval_metric='rmse', early_stopping_rounds=10)
FLAMES_xgb.fit(X_train, y_train, eval_set=eval_set, verbose=True)


# Calibrate XGB & PoPS integration

In [ ]:
xdf = df.drop_duplicates(keep="first")
xdf = xdf.drop_duplicates(subset=["ensg", "symbol", 'filename'], keep="first")
new_df=[]
for chrom in range(1,23):    
    tdf = xdf[xdf['chr'] == chrom].copy()
    X_train = tdf[feats]
    y = [x[1] for x in LOCO_models[chrom-1].predict_proba(X_train)]
#     y=  [x[1] for x in FLAMES_xgb.predict_proba(X_train)]
    tdf['XGB_score'] = y
    new_df.append(tdf)
results = pd.concat(new_df)

print(len(set(results['filename'])), sum(results['TP']))
best_x = []
best_y =[]
best = 0
for x in np.linspace(0,0.9999999, 5):
    print(np.round(x, 2))
    for y in np.linspace(0,2,5):
        test_results = results.groupby('filename').apply(Flames_scoring_light, x, y) 
        if sum(test_results['TP'] * test_results['FLAMES_causal'])/sum(test_results['FLAMES_causal']) > best:
            best = sum(test_results['TP'] * test_results['FLAMES_causal'])/sum(test_results['FLAMES_causal'])
            best_combi = (np.round(x,2),np.round(y,2))
print(best_combi)
            
for x in np.linspace(best_combi[0] - 0.125,best_combi[0] + 0.125, 5):
    print(np.round(x, 4))
    for y in np.linspace(best_combi[1]-0.25,best_combi[1]+0.25,5):
        test_results = results.groupby('filename').apply(Flames_scoring_light, x, y) 
        if sum(test_results['TP'] * test_results['FLAMES_causal'])/sum(test_results['FLAMES_causal']) > best:
            best = sum(test_results['TP'] * test_results['FLAMES_causal'])/sum(test_results['FLAMES_causal'])
            best_x = [x]
            best_y = [y]
        elif sum(test_results['TP'] * test_results['FLAMES_causal'])/sum(test_results['FLAMES_causal']) == best:
            best = sum(test_results['TP'] * test_results['FLAMES_causal'])/sum(test_results['FLAMES_causal'])
            best_x.append(x)
            best_y.append(y)
print(np.mean(best_x), np.mean(best_y))

# Calibrate FLAMES framework

In [36]:
df = pd.read_csv(os.path.join('benchmarking_results', 'calibration', 'calibration.tsv'), sep='\t')
ts = []
rs = []
recs = []
for i in range(1000):
    best_r = 0
    thresh = 0
    min_raw = 0
    sampledf = df.sample(replace=True, frac=1, random_state=i)
    for x in np.linspace(1,0,26):
        tempdf = sampledf[sampledf['FLAMES_causal'] == 1]
        tempdf = tempdf[tempdf['FLAMES_raw'] > x]
        tempdf = tempdf.sort_values('FLAMES_scaled', ascending= False)
        if len(tempdf) == 0:
            continue
        cumsum = np.cumsum(tempdf['TP'])
        rank = np.arange(len(cumsum)) + 1
        Num = sum(sampledf['TP'])
        precisions = cumsum / rank
        recalls = cumsum / Num
        last = 0
        for i in range(len(precisions)):
            if list(precisions)[i] >= 0.75:
                last = i
        i = last
        precision_at_threshold = list(precisions)[i]
        recall_at_threshold = list(recalls)[i]
        if recall_at_threshold > best_r:
            best_r = recall_at_threshold
            thresh = list(tempdf['FLAMES_scaled'])[i]
            min_raw = np.round(x,2)
    ts.append(thresh)
    rs.append(min_raw)
    recs.append(best_r)
print(f'Scaled score threshold:{np.round(np.mean(ts),3)}\nRaw score threshold: {np.round(np.mean(rs),3)}\nEstimated recall at thresholds: {np.round(np.mean(recs),3)}')

Scaled score threshold:0.248
Raw score threshold: 0.136
Estimated recall at thresholds: 0.364


In [28]:
df = pd.read_csv(os.path.join('benchmarking_results', 'calibration', 'calibration.tsv'), sep='\t')
ts = []
recs = []
bsdf = []
for i in range(1000):
    best_r = 0
    thresh = 0
    min_raw = 0
    sampledf = df.sample(replace=True, frac=1, random_state=i)
    tempdf = sampledf[sampledf['FLAMES_causal'] == 1]
    tempdf = tempdf.sort_values('FLAMES_raw', ascending= False)
    cumsum = np.cumsum(tempdf['TP'])
    rank = np.arange(len(cumsum)) + 1
    Num = sum(sampledf['TP'])
    precisions = cumsum / rank
    recalls = cumsum / Num
    last = 0
    bsdf.append(tempdf)
    for i in range(len(precisions)):
        if list(precisions)[i] >= 0.75:
            last = i
    i = last
    precision_at_threshold = list(precisions)[i]
    recall_at_threshold = list(recalls)[i]
    if recall_at_threshold > best_r:
        best_r = recall_at_threshold
        thresh = list(tempdf['FLAMES_scaled'])[i]
    ts.append(thresh)
    recs.append(best_r)
print(f'Raw score threshold:{np.round(np.mean(ts),3)}\nEstimated recall at thresholds: {np.round(np.mean(recs),3)}')

Raw score threshold:0.307
Estimated recall at thresholds: 0.273
